<a href="https://colab.research.google.com/github/aditi2306/Principles/blob/main/FineTune_NER_FinBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch


In [2]:
!pip install flair

In [3]:
!pip install datasets

In [4]:
!pip install numpy==1.25.2
!pip install --upgrade transformers
!pip install --upgrade scikit-learn

  Using cached transformers-4.50.2-py3-none-any.whl.metadata (39 kB)
Using cached transformers-4.50.2-py3-none-any.whl (10.2 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0


In [8]:
!pip install flair # Make sure flair is installed

In [11]:
import re
import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from flair.data import Sentence
from flair.models import SequenceTagger
import re
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from flair.data import Sentence
from flair.models import SequenceTagger
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"

In [12]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

# Check dataset structure
print(dataset)
print(dataset['train'][0])  # Print first record


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9543
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2388
    })
})
{'text': '$BYND - JPMorgan reels in expectations on Beyond Meat https://t.co/bd0xbFGjkT', 'label': 0}


In [13]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

In [14]:
model_name ='ProsusAI/finbert'

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)


In [9]:
ner_tagger = SequenceTagger.load("flair/ner-english")

pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

2025-03-27 20:08:45,497 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:


# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Function to extract entities using Flair NER
def extract_entities_batch(texts):
    sentences = [Sentence(text) for text in texts]  # Convert texts to Flair sentences
    ner_tagger.predict(sentences)  # Batch process
    return [[entity.text for entity in sentence.get_spans("ner") if entity.tag == "ORG"] for sentence in sentences]

# Function to preprocess and tokenize dataset
def preprocess_function(examples):
    cleaned_texts = [clean_text(t) for t in examples["text"]]

    # Use batch entity extraction instead of single entity extraction
    entity_texts = extract_entities_batch(cleaned_texts)  # Extract entities in batch

    # Combine extracted entities with cleaned text
    combined_texts = [f"{' '.join(entities)} {text}" if entities else text for text, entities in zip(cleaned_texts, entity_texts)]

    return tokenizer(combined_texts, padding="max_length", truncation=True)

# Apply preprocessing to dataset
tokenized_train = dataset["train"].map(preprocess_function, batched=True)
tokenized_validation = dataset["validation"].map(preprocess_function, batched=True)

# Extract tweets for visualization (sample from dataset)
sample_tweets = dataset["train"]["text"][:20]  # Taking first 20 tweets for visualization

# Process tweets for visualization
cleaned_texts = [clean_text(tweet) for tweet in sample_tweets]
entities = extract_entities_batch(sample_tweets)  # Corrected function usage

# Flatten entity list
all_entities = [ent for sublist in entities for ent in sublist]


# Generate Word Cloud (Before & After Cleaning)
def generate_wordcloud(texts, title):
    wordcloud = WordCloud(width=600, height=400, background_color="white").generate(" ".join(texts))
    plt.figure(figsize=(8, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title)
    plt.show()

# Generate Bar Chart for Entities
def plot_entity_frequencies(entities):
    entity_counts = Counter(entities)
    plt.figure(figsize=(8, 5))
    plt.bar(entity_counts.keys(), entity_counts.values(), color="skyblue")
    plt.xticks(rotation=45)
    plt.xlabel("Entities")
    plt.ylabel("Frequency")
    plt.title("Most Extracted Financial Entities")
    plt.show()

# Visualization
generate_wordcloud(sample_tweets, "Before Cleaning")
generate_wordcloud(cleaned_texts, "After Cleaning")
plot_entity_frequencies(all_entities)


Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

2025-03-27 22:40:55,375 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2025-03-27 22:41:40,794 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2025-03-27 22:41:40,795 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2025-03-27 22:41:40,796 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

# Create a custom dataset for PyTorch
class FinancialNewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }

# Prepare the dataset with additional NER entity features
def prepare_dataset_with_ner(dataset, tokenizer):
    texts = dataset["text"]  # Text data
    labels = dataset["label"]  # Labels (you should have sentiment labels)

    # Apply NER extraction and combine the results with the cleaned text
    entity_texts = []
    for text in texts:
        # Extract entities using NER
        entities = extract_entities(text)
        entity_str = " ".join(entities)

        # Combine the entities with the original text
        enhanced_text = f"{entity_str} {text}" if entity_str else text
        entity_texts.append(enhanced_text)

    return entity_texts, labels

def compute_metrics(p):
    predictions = torch.argmax(torch.tensor(p.predictions), dim=-1).numpy()  # Convert logits to predictions
    return {"accuracy": accuracy_score(p.label_ids, predictions)}

# Prepare the training and validation datasets
train_texts, train_labels = prepare_dataset_with_ner(dataset["train"], tokenizer)
val_texts, val_labels = prepare_dataset_with_ner(dataset["validation"], tokenizer)

# Create DataLoader for PyTorch
train_dataset = FinancialNewsDataset(train_texts, train_labels, tokenizer)
val_dataset = FinancialNewsDataset(val_texts, val_labels, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./finbert_output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=200,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,  # 🔥 Adds dynamic padding
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-179881fc8256>:77: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 0.7277, 'grad_norm': 12.380261421203613, 'learning_rate': 4.720592344230232e-05, 'epoch': 0.16764459346186086}
{'loss': 0.5334, 'grad_norm': 9.564476013183594, 'learning_rate': 4.441184688460464e-05, 'epoch': 0.3352891869237217}
{'loss': 0.4817, 'grad_norm': 10.237343788146973, 'learning_rate': 4.1617770326906956e-05, 'epoch': 0.5029337803855826}
{'loss': 0.4528, 'grad_norm': 20.77789878845215, 'learning_rate': 3.882369376920928e-05, 'epoch': 0.6705783738474435}
{'loss': 0.4536, 'grad_norm': 5.577925205230713, 'learning_rate': 3.60296172115116e-05, 'epoch': 0.8382229673093042}
{'eval_loss': 0.38653212785720825, 'eval_accuracy': 0.8672529313232831, 'eval_runtime': 72.4316, 'eval_samples_per_second': 32.969, 'eval_steps_per_second': 4.128, 'epoch': 1.0}
{'loss': 0.4096, 'grad_norm': 12.216562271118164, 'learning_rate': 3.323554065381392e-05, 'epoch': 1.0058675607711651}
{'loss': 0.2576, 'grad_norm': 0.17734210193157196, 'learning_rate': 3.0441464096116235e-05, 'epoch': 1.1735121

TrainOutput(global_step=3579, training_loss=0.2999884804456252, metrics={'train_runtime': 2954.7767, 'train_samples_per_second': 9.689, 'train_steps_per_second': 1.211, 'train_loss': 0.2999884804456252, 'epoch': 3.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.5968661904335022, 'eval_accuracy': 0.8865159128978225, 'eval_runtime': 72.0193, 'eval_samples_per_second': 33.158, 'eval_steps_per_second': 4.152, 'epoch': 3.0}
{'eval_loss': 0.5968661904335022, 'eval_accuracy': 0.8865159128978225, 'eval_runtime': 72.0193, 'eval_samples_per_second': 33.158, 'eval_steps_per_second': 4.152, 'epoch': 3.0}


In [ ]:
# Function to predict sentiment and extract company names
def predict_sentiment_and_entities(texts):
    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Move inputs to the same device as the model
    device = next(model.parameters()).device  # Get the model's device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Make predictions using the trained model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class (the label with the highest score)
    predictions = torch.argmax(outputs.logits, dim=-1)

    # Extract company names using NER
    company_names = []
    for text in texts:
        entities = extract_entities(text)  # Extract company names (ORG entities)
        company_names.append(entities)

    # Return company names and predictions
    return company_names, predictions

# Example: Make predictions on new data
new_texts = [
    "Apple's stock is skyrocketing after the earnings report.",
    "Tesla faces production delays which might hurt its stock price."
]

# Get company names and predictions for the new texts
company_names, predictions = predict_sentiment_and_entities(new_texts)

# Print company names and corresponding sentiment predictions
for i, text in enumerate(new_texts):
    sentiment = predictions[i].item()  # Convert tensor to scalar value
    company = ", ".join(company_names[i]) if company_names[i] else "No company mentioned"
    sentiment_label = ["Negative", "Neutral", "Positive"][sentiment]
    print(f"Text: {text}")
    print(f"Company: {company}")
    print(f"Sentiment: {sentiment_label}\n")


Text: Apple's stock is skyrocketing after the earnings report.
Company: Apple
Sentiment: Negative

Text: Tesla faces production delays which might hurt its stock price.
Company: Tesla
Sentiment: Negative

